In [ ]:
#%pip install geopandas

In [ ]:
%pip install earthengine-api geemap geedim pandas geopandas pycrs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.4 MB/s eta 0:00:00
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=a13981301ed9486a379cf8dcb2bee5d9cae4150c688a9ba19ede73418b4c50d2
  Stored in directory: /root/.cache/pip/wheels/5f/ad/a3/183ed754d7698fc15a2eb153705e05d05a0d97f3331293ce48
Successfully built pycrs


In [ ]:
# Import the needed libraries
import ee
import geemap
import geedim as gd
import pandas as pd
import geopandas as gpd
import os

In [ ]:
# Trigger the authentication flow
ee.Authenticate()


# Initialize the library
ee.Initialize(project="ee-kingluda357")

# Initializes geemap/ GEE Python API
geemap.ee_initialize()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Admin0.zip to Admin0.zip


In [ ]:
import zipfile
import os

with zipfile.ZipFile("Admin0.zip", 'r') as zip_ref:
    zip_ref.extractall("shapefile")

In [ ]:
import geopandas as gpd

shapefile_path = "/content/shapefile/Admin0"
gdf = gpd.read_file(shapefile_path)
gdf.head()

,fid,geometry
0,1.0,"POLYGON ((-0.40564 5.47005, -0.40589 5.46979, ..."


In [ ]:
# Function to visualize a FeatureCollection
def feature_map(feature, vis_params, name):
    Map = geemap.Map(center=[8.64, 18.05], zoom =3)
    Map.add_basemap("SATELLITE")

    styled_layer = feature.style(**vis_params)

    Map.add_layer(styled_layer, {}, name)
    return Map


# Function to visualize a EE Image/Raster
def raster_map(raster, viz_params, name):
    """
    """

    Map = geemap.Map(center=[8.64, 18.05], zoom =3)
    Map.add_basemap("SATELLITE")
    Map.add_layer(raster, viz_params, name)
    return Map

# Defining the ROI

In [ ]:
# Load shapefile and convert to EE FeatureCollection
def boundary_to_fc(shp_path):
    fc = geemap.shp_to_ee(shp_path)
    return fc


# Load and dissolve
shp_path = "/content/shapefile/Admin0/Ghana.shp"
fc = boundary_to_fc(shp_path)

# Show on map
Map = geemap.Map()
Map.addLayer(fc, {"color": "blue", "fillColor": "00000000", "width": 2}, "Dissolved Boundary")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Aggregating the Population Layer

In [ ]:
# GHSL: Global Population Surfaces Image Collection
# GHSL: Spatial distribution of residential population. Population Count.
# Reference: https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP#description
ghsl_pop_collection = (ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")
            .filterDate("2000", "2025")
            .filterBounds(fc))


# Check the native projection of the GHS pop layer
ghsl_native_projection = ghsl_pop_collection.first().projection()
ghsl_native_resolution = ghsl_native_projection.nominalScale().getInfo()

#Checking the Projection and Resolution
print("The Native Projection of the GHSL Pop Layer: ", ghsl_native_projection.getInfo())
print("The Native Resolution/Scale of the GHSL Pop Layer: ", ghsl_native_resolution)

The Native Projection of the GHSL Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [100, 0, -18041000, 0, -100, 9000000]}
The Native Resolution/Scale of the GHSL Pop Layer:  100


# Downloading Population Data for Specific Years

In [ ]:
# Direct download of GHS pop data for specific year (2000 -2025)
ghsl_pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2025").clip(fc) # Change to your desired year. The data is available at 5-years interval.

# Inspect the date of the GHS layer
ghsl_pop_date = ee.Date(ghsl_pop.get("system:time_start")).format("YYYY-MM-dd").getInfo()
print(f"The date of the GHSL pop layer is {ghsl_pop_date}")

The date of the GHSL pop layer is 2025-01-01


# Aggregating at a lower Resolution

In [ ]:
# Aggregate the GHS population layer to desired lower resolution
def ghs_resolution_aggregate(pop_image, native_proj, new_resolution):

    # Get the projection at the desired scale
    ghs_projection_at_new_resolution = native_proj.atScale(new_resolution)

    ghs_pop_at_new_res = pop_image.reduceResolution(
        reducer = ee.Reducer.sum().unweighted(),
        maxPixels = 1024
    ).reproject(
        crs = ghs_projection_at_new_resolution #Request the data at the scale and projection of reduced resolution (1km)
    )

    # Inspect the resolution/scale of the 'ghs_pop_at_new_res'
    ghs_pop_at_new_res_projection = ghs_pop_at_new_res.projection()
    ghs_pop_at_new_res_resolution = ghs_pop_at_new_res_projection.nominalScale().getInfo()

    print("The Native Projection of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_projection.getInfo())
    print("The Resolution/Scale of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_resolution)

    return ghs_pop_at_new_res


# Apply function
ghsl_pop_1km = ghs_resolution_aggregate(ghsl_pop, ghsl_native_projection, 1000)

The Native Projection of the Aggregated GHS Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [1000, 0, -18041000, 0, -1000, 9000000]}
The Resolution/Scale of the Aggregated GHS Pop Layer:  1000


# Displaying and Visualizing Population Layer

In [ ]:
# Display the population layer
# Visualization parameters for the GHS pop layer
vis_params_ghs = {
  "bands" : ["population_count"],
  "min" : 0.0,
  "max": 50.0,
  "palette" : ["ffffe7", "FFc869", "ffac1d", "e17735", "f2552c", "9f0c21", "9f0c21"]
}

# Add the GHS pop layer to the map and visualize
pop_map = raster_map(ghsl_pop.clip(fc), vis_params_ghs , "GHSL Population 2025 (100m)")
pop_map.addLayer(ghsl_pop_1km.clip(fc), vis_params_ghs , "GHSL Population 2025 (1km)")
pop_map

Map(center=[8.64, 18.05], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

# Exporting Map

In [ ]:
# Function to export image to Google Drive
def img_export_to_drive(image, desc, aoi, scale, folder_name):
    img_Export = ee.batch.Export.image.toDrive(
        image = image,
        description = desc,
        folder = folder_name,
        region = aoi,
        #crs = "EPSG:32634",
        scale = scale,
        maxPixels = 1e13
        )

    img_Export.start()

    return img_Export


# Export the Original GHSL population layer
#ghsl_pop_export = img_export_to_drive(ghsl_pop.clip(roi), "GHSL_Pop_Africa_2025_100m", roi, 100, "QGIS_Maps_2025")

# Export the GHSL population layer
ghsl_pop_export = img_export_to_drive(ghsl_pop_1km.clip(fc), "GHSL_Pop_Africa_2025_1km", fc.geometry(), 1000, "2025_Pop")